# Generate data for `test_correct_count_rate`

In [ ]:
import importlib
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer.buildfgssteps import BuildFGSSteps, shift_to_id_attitude

In [ ]:
def find_package_location(package):
    return importlib.util.find_spec(package).submodule_search_locations[0]


def find_repo_location(package):
    return os.path.abspath(os.path.join(find_package_location(package), os.pardir))

In [ ]:
repo_location = find_repo_location(package='jwst_magic')
__location__ = os.path.join(repo_location, 'jwst_magic', 'tests')
ROOT = 'test_buildfgssteps'
SEGMENT_INFILE = os.path.join(__location__, 'data', 'unshifted_all_found_psfs_{}_G1.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))
CENTER_POINTING_1 = os.path.join(__location__, 'data', 'center_pointing_{}_G1.txt'.format(ROOT))
TEST_DIRECTORY = os.path.join(__location__, 'out', ROOT)

In [ ]:
with fits.open( os.path.join(__location__, 'data', 'fgs_data_2_cmimf.fits')) as hdulist:
    open_image = hdulist[1].data

In [ ]:
d = {}

for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 3
        else:
            n = 1
            
        for i in range(n):
            # Add random seed to match what's in the test_correct_count_rate test
            np.random.seed(100)
            
            # Run the code
            fgs_im, guiding_selections_file, psf_center_file = shift_to_id_attitude(
                    open_image, ROOT, guider, TEST_DIRECTORY, guiding_selections_file=SELECTED_SEGS,
                    all_found_psfs_file=SEGMENT_INFILE, center_pointing_file=CENTER_POINTING_1,
            psf_center_file=None, logger_passed=True)
            BFS = BuildFGSSteps(fgs_im, guider, ROOT, step, guiding_selections_file=guiding_selections_file,
                                out_dir=TEST_DIRECTORY, shift_id_attitude=True, use_readnoise=False)

            # Don't know how to test bias - since it is so variable
            if step != 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step == 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , step, 'countrate']):
            l = np.array(l)
            if (l.size > 0 and step == 'LOSTRK') or (name in ['norm']):
                d[guider][step][name] = l[0]
            elif name == 'countrate':
                d[guider][step][name] = list(l)
            elif l.size > 0 and name != 'countrate':
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]

        

In [ ]:
# What is printed here is used in jwst_magic/tests/data/parameterized_test_data.yaml
pprint.pprint(d)

In [ ]:
# print out data the same way it is in the .yaml file
mode_order = ['CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK', 'LOSTRK']
for k in d:
    print(f"guider{k}:")
    for mode in mode_order:
        print(f"\t{mode}:")
        print(f"\t\tcountrates: {d[k][mode]['countrate']}")
        if mode != 'LOSTRK':
            print(f"\t\tbias: {d[k][mode]['bias']}")
        print(f"\t\ttime_normed_im: {d[k][mode]['norm']}")
        if mode not in ['LOSTRK', 'TRK']:
            print(f"\t\tcds: {d[k][mode]['cds']}")
        if mode == 'ID':
            print(f"\t\tstrips: {d[k][mode]['strips']}")
        print(f"\t\t{mode}: {d[k][mode][mode]}")
        

# Generate data for `coordinate_tranforms`

In [ ]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [ ]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        elif 'detector' in arg_names:
            result = func_to_call(x, y, 'NRCA3')
            print(result)
        else:
            result = func_to_call(x, y)
            print(result)

In [ ]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)

# Generate data for `test_generate_segment_override_file`

In [ ]:
from datetime import datetime

from jwst_magic.tests.utils import parametrized_data
from jwst_magic.utils import utils
from jwst_magic.segment_guiding.segment_guiding import (generate_segment_override_file,
                                                        SegmentGuidingCalculator,
                                                        generate_photometry_override_file)

In [ ]:
repo_location = find_repo_location(package='jwst_magic')
__location__ = os.path.join(repo_location, 'jwst_magic', 'tests')
ROOT = "test_sgt"
SEGMENT_INFILE = os.path.join(__location__, 'data', '{}_ALLpsfs.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))
SELECTED_SEGS2 = os.path.join(__location__, 'data', 'unshifted_guiding_selections_{}_G1_config1.txt'.format(ROOT))
SELECTED_SEGS3 = os.path.join(__location__, 'data', 'unshifted_guiding_selections_{}_G1_config2.txt'.format(ROOT))
SHIFTED_SEGS = os.path.join(__location__, 'data', 'shifted_guiding_selections_{}_G1_config1.txt'.format(ROOT))
SHIFTED_SEGS2 = os.path.join(__location__, 'data', 'shifted_guiding_selections_{}_G1_config2.txt'.format(ROOT))
SHIFTED_SEGS3 = os.path.join(__location__, 'data', 'shifted_guiding_selections_{}_G1_config3.txt'.format(ROOT))
SHIFTED_INFILE = os.path.join(__location__, 'data', 'shifted_all_found_psfs_{}_G1_config1.txt'.format(ROOT))
SHIFTED_INFILE2 = os.path.join(__location__, 'data', 'shifted_all_found_psfs_{}_G1_config2.txt'.format(ROOT))
SHIFTED_INFILE3 = os.path.join(__location__, 'data', 'shifted_all_found_psfs_{}_G1_config3.txt'.format(ROOT))

In [ ]:
PROGRAM_ID = 1141
OBSERVATION_NUM = 7
VISIT_NUM = 1
test_directory = os.path.join(__location__, 'out', ROOT)

In [ ]:
sof_parameters = [
                  ([0,0], [SELECTED_SEGS], [SEGMENT_INFILE], test_data[0]),
                  ([4], [SELECTED_SEGS], [SEGMENT_INFILE], test_data[1]),
                  ([0,0], [SELECTED_SEGS, SELECTED_SEGS2], [SEGMENT_INFILE, SEGMENT_INFILE], test_data[2]), # no matching segs
                  (0, [SELECTED_SEGS, SELECTED_SEGS3], [SEGMENT_INFILE, SEGMENT_INFILE], test_data[3]), # 2 matching segs
                  (0, np.array([[1, 12, 6]], dtype=object), [SEGMENT_INFILE], test_data[4]),
                  (0, np.array([[1, 2, 3], [4, 1, 18, 12, 2]], dtype=object), [SEGMENT_INFILE, SEGMENT_INFILE], test_data[5]),
                  (0, [SHIFTED_SEGS, SHIFTED_SEGS2], [SHIFTED_INFILE, SHIFTED_INFILE2], test_data[6]), # no matching segs
                  (0, [SHIFTED_SEGS, SHIFTED_SEGS3], [SHIFTED_INFILE, SHIFTED_INFILE3], test_data[7]), # matching guide star
                 ]
for (center_of_pointing, selected_segs, segment_infile, correct_command) in sof_parameters:

    guider = 1

    guide_star_params_dict = {'v2_boff': 0.1,
                              'v3_boff': 0.2,
                              'fgs_num': guider,
                              'ra': 90.9708,
                              'dec': -67.3578,
                              'pa': 157.1234,
                              'center_of_pointing': center_of_pointing}

    generate_segment_override_file(
        segment_infile, guider, PROGRAM_ID, OBSERVATION_NUM, VISIT_NUM, root=ROOT,
        out_dir=__location__, selected_segs_list=selected_segs,
        guide_star_params_dict=guide_star_params_dict, parameter_dialog=False
    )

    # Check to make sure the override file was created, and in the right place
    segment_override_file = os.path.join(test_directory, '{}_gs_override_1141_7_1.txt'.format(
        datetime.now().strftime('%Y%m%d')))
    assert os.path.isfile(segment_override_file)

    # # Check to make sure the command was written correctly
    with open(segment_override_file) as f:
        segment_override_command = f.read()
    
    print(f'"{segment_override_command}", \n')


# Generate data for `test_write_override_report`

In [ ]:
guider = 1
guide_star_params_dict = {'v2_boff': 0.1,
                          'v3_boff': 0.2,
                          'fgs_num': guider,
                          'ra': 90.9708,
                          'dec': -67.3578,
                          'pa': 157.1234,
                          'center_of_pointing': 0}

# Multiple commands - so multiple segment infiles
selected_segs = np.array([[1, 2, 3], [4, 1, 18, 12, 2]], dtype=object)
segment_infile = [SEGMENT_INFILE, SEGMENT_INFILE]

generate_segment_override_file(
    segment_infile, guider, PROGRAM_ID, OBSERVATION_NUM, VISIT_NUM, root=ROOT,
    out_dir=__location__, selected_segs_list=selected_segs,
    guide_star_params_dict=guide_star_params_dict, parameter_dialog=False
)

report_file = '{}_gs_override_{}_{}_{}_REPORT.txt'.format(datetime.now().strftime('%Y%m%d'),
                                                          PROGRAM_ID,
                                                          OBSERVATION_NUM,
                                                          VISIT_NUM)
report_file = os.path.join(test_directory, report_file)

with open(report_file) as f:
    lines = f.read().split('\n')
    print(lines)

In [ ]:
report_file

# Generate data for `test_segment_override_file_single_obs`

In [ ]:
test_data = PARAMETRIZED_DATA['test_segment_override_file_single_obs']
obs_num, correct_file_name, correct_command = '2', 'gs_override_1141_2_1.txt', test_data[0]

In [ ]:
guider = 1
guide_star_params_dict = {'v2_boff': 0.1,
                          'v3_boff': 0.2,
                          'fgs_num': guider,
                          'ra': 90.9708,
                          'dec': -67.3578,
                          'pa': 157.1234,
                          'center_of_pointing': 0}

generate_segment_override_file(
    [SEGMENT_INFILE], guider, PROGRAM_ID, obs_num, 1, root=ROOT,
    out_dir=__location__, selected_segs_list=[SELECTED_SEGS],
    guide_star_params_dict=guide_star_params_dict, parameter_dialog=False
)

# Check to make sure the override file was created, and in the right place
segment_override_file = os.path.join(
    test_directory, '{}_{}'.format(datetime.now().strftime('%Y%m%d'), correct_file_name)
)
assert os.path.isfile(segment_override_file)

# Also check for the report
segment_override_report = os.path.join(
    test_directory, '{}_{}'.format(datetime.now().strftime('%Y%m%d'),
                                   correct_file_name.split('.txt')[0] + '_REPORT.txt')
)
assert os.path.isfile(segment_override_report)

# # Check to make sure the command was written correctly
with open(segment_override_file) as f:
    segment_override_command = f.read()

In [ ]:
segment_override_command